In [51]:
from collections import defaultdict
import pandas as pd
import polyline as pl


df = pd.read_csv("../data/train.csv/train.csv")


In [30]:
#df_test = pd.read_csv("../data/train.csv/train.csv")
df_test = pd.read_csv("../data/test_additional.csv")
#df_test = pd.read_csv("../data/validation.csv")



In [38]:
epsilon = 0.0001

class PointList:
    def __init__(self):
        self.coord = []
        self.accm = []
        self.cntr = []
        #self.type = []
        self.numPoints = 0
        self.dict = defaultdict(lambda: defaultdict(list))
        
    def addlist(self, point, weight = 1):
        closest = -1
        ix = int(point[0] * 1000)
        iy = int(point[1] * 1000)
        for i in self.dict[ix][iy]:
            dx = abs(self.coord[i][0] - point[0]) + abs(self.coord[i][1] - point[1])
            if dx < epsilon:
                closest = i
                self.accm[i][0] += point[0] * weight
                self.accm[i][1] += point[1] * weight
                self.cntr[i] += weight
                self.coord[i][0] = self.accm[i][0] / self.cntr[i]
                self.coord[i][1] = self.accm[i][1] / self.cntr[i]
                break
        if closest == -1:
            self.dict[ix][iy].append(self.numPoints)
            self.coord.append([point[0], point[1]])
            self.accm.append([point[0]*weight, point[1]*weight])
            self.cntr.append(weight)
            #self.type.append(type)
            closest = self.numPoints
            self.numPoints += 1
        return closest
    
    def findpoint(self, point):
        closest = -1
        ix = int(point[0] * 1000)
        iy = int(point[1] * 1000)
        for i in self.dict[ix][iy]:
            dx = abs(self.coord[i][0] - point[0]) + abs(self.coord[i][1] - point[1])
            if dx < epsilon and self.cntr[i] > 4:
                closest = i
                break
        return closest
    
    def getlinepoints(self, my_list, l, r, epsi = 0.0001):
        if l[0] == r[0] and l[1] == r[1]:
            c  = self.findpoint(l)
            if c != -1:
                my_list.append(c)
            return
        lx = int(l[0] * 1000)
        ly = int(l[1] * 1000)
        rx = int(r[0] * 1000)
        ry = int(r[1] * 1000)
        x1 = min(lx, rx)
        x2 = max(lx, rx)
        y1 = min(ly, ry)
        y2 = max(ly, ry)
        xx = l[0]
        yy = l[1]
       
        vx = r[0] - l[0]
        vy = r[1] - l[1]
        
        la = vy
        lb = -vx
        nrm = math.sqrt(la*la+lb*lb)
        la /= nrm
        lb /= nrm
        lc = -la * xx - lb * yy
        
        templist = []
        for x in range(x1, x2+1):
            for y in range(y1, y2+1):
                for i in self.dict[x][y]:
                    d = abs(la*self.coord[i][0]+lb*self.coord[i][1]+lc)
                    if d > epsi:
                        continue
                    px = self.coord[i][0] - l[0]
                    py = self.coord[i][1] - l[1]
                    
                    pos = px * -lb + py * la
                    
                    if 0.0 <= pos <= nrm:
                        templist.append([i, pos])
        templist = sorted(templist, key = lambda k : k[1])
        
        #print(templist)
        
        for x in templist:
            my_list.append(x[0])
        
#    def getlinepoints(self, my_list, l, r, epsi = epsilon):
#        if l[0] == r[0] and l[1] == r[1]:
#            c  = self.findpoint(l)
#            if c != -1:
#                my_list.append(c)
#            return
#        lx = int(l[0] * 1000)
#        ly = int(l[1] * 1000)
#        rx = int(r[0] * 1000)
#        ry = int(r[1] * 1000)
#        if lx < rx:
#            dx = 1
#        elif lx == rx:
#            dx = 0
#        else:
#            dx = -1
#        if ly < ry:
#            dy = 1
#        elif ly == ry:
#            dy = 0
#        else:
#            dy = -1
#            
#        x = l[0]
#        y = l[1]
#       
#        vx = r[0] - l[0]
#        vy = r[1] - l[1]
#        
#        la = vy
#        lb = -vx
#        nrm = math.sqrt(la*la+lb*lb)
#        la /= nrm
#        lb /= nrm
#        lc = -la * x - lb * y
#        
#        templist = []
#        
#        count = 0
#        
#        print(vx, vy,  dx, dy)
#        
#        while True:
#           for i in self.dict[lx][ly]:
#               d = abs(la*self.coord[i][0]+lb*self.coord[i][1]+lc)
#               if d > epsi:
#                    continue
#                px = self.coord[i][0] - l[0]
#                py = self.coord[i][1] - l[1]
#                
#               pos = px * -lb + py * la
#                if 0.0-epsilon <= pos <= nrm+epsilon:
#                    templist.append([i, pos])
#            count += 1
#            if (count > 100):
#                break
#            if lx == rx and ly == ry:
#                break
#            print(x, y, lx, ly, rx, ry)
#            if vx == 0:
#                ly = ly + dy
#                y += dy * 0.001
#                continue
#            if vy == 0:
#                lx = lx + dx
#                x += dx * 0.001
#                continue
#            nx = lx + dx
#            ty = y + vy * (nx * 0.001 - x) / vx
#            ny = ly + dy
#            tx = x + vx * (ny * 0.001 - y) / vy
#            d1 = (nx * 0.001-x)*(nx * 0.001-x)+(ty-y)*(ty-y)
#            d2 = (tx-x)*(tx-x)+(ny * 0.001-y)*(ny * 0.001-y)
#            if d1 < d2:
#                x = nx * 0.001
#                y = ty
#                lx = nx
#            else:
#                x = tx
#                y = ny * 0.001
#                ly = ny
#        sorted(templist, key = lambda k : k[1])
#        
#        for x in templist:
#            my_list.append(x[0])
    
    def plot(self, my_map):
        for i in range(self.numPoints):
            if (self.cntr[i] <= 4):
                continue
            x = self.coord[i]
            #if self.type[i] == 0:
            #    col = "blue"
            #else:
            #    col = "green"
            
            col = "blue"
            folium.CircleMarker(location=[x[0], x[1]], color=col, radius = 8).add_to(my_map)



In [52]:
import numpy as np
from collections import defaultdict
import math

citylist = defaultdict(list)

ROWS = len(df)
#ROWS = 1000

#ff = open('routedist.txt', 'w')

for i in range(ROWS):
    rw = df.iloc[i]
    #if (pd.isna(rw.track)):
    #    print("Skipped {}".format(id))
    #    continue
    if (i % 1000) == 0:
        print(i//1000)
    city = rw.main_id_locality
    citylist[city].append(i)
    #routePoints = pl.decode(rw.route)
    #numPoints = len(routePoints)
    #routeLength = 0.0
    #for j in range(0, numPoints-1):
    #    dx = routePoints[j+1][0] - routePoints[j][0]
    #    dy = routePoints[j+1][1] - routePoints[j][1]
    #    routeLength += math.sqrt(dx*dx+dy*dy)
    #print(routeLength,file = ff)


#print(citylist)

#ff.close()

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [67]:
df.iloc[831223].Id

825343

In [31]:
import numpy as np
from collections import defaultdict
import math

citylist_test = defaultdict(list)

ROWS = len(df_test)
#ROWS = 1000

#ff = open('routedist.txt', 'w')

for i in range(ROWS):
    rw = df_test.iloc[i]
    #if (pd.isna(rw.track)):
    #    print("Skipped {}".format(id))
    #    continue
    if (i % 1000) == 0:
        print(i//1000)
    city = rw.main_id_locality
    citylist_test[city].append(i)
    #routePoints = pl.decode(rw.route)
    #numPoints = len(routePoints)
    #routeLength = 0.0
    #for j in range(0, numPoints-1):
    #    dx = routePoints[j+1][0] - routePoints[j][0]
    #    dy = routePoints[j+1][1] - routePoints[j][1]
    #    routeLength += math.sqrt(dx*dx+dy*dy)
    #print(routeLength,file = ff)


#print(citylist)

#ff.close()


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89


In [49]:
# cities: 1078, 22390, 22430, 22438
# cities old: 338, 22402, 22406, 22394

from _collections import defaultdict
import random
import folium

epsilon = 0.0001

city = 22394

start = citylist[city][0]

dat = pl.decode(df.iloc[start].route)
sp = dat[0]

my_map = folium.Map(sp, zoom_start=20)
#for i in citylist[city]:
#    dat = pl.decode(df.iloc[i].route)
#    #print(dat[0])
#    folium.PolyLine(dat, color="red", weight=2.5, opacity=1).add_to(my_map)

plst = PointList()

count = 0

for i in citylist[city]:
    #if (plst.numPoints >= 15000):
    #    break
    count += 1
    if (count % 1000  == 0):
        print(count // 1000,  plst.numPoints)
    try:
        dat = pl.decode(df.iloc[i].route)
    except TypeError:
        print("invalid route", i)
        continue
    #folium.PolyLine(dat, color="red", weight=2.5, opacity=1).add_to(my_map)
    numPoints = len(dat)
    for j in range(0, numPoints):
        if j == 0 or j == numPoints-1:
            # plst.addlist(dat[j], 1)
            continue
        doadd = 1
        dx1 = -dat[j-1][0] + dat[j][0]
        dy1 = -dat[j-1][1] + dat[j][1]
        dx2 = dat[j+1][0] - dat[j][0]
        dy2 = dat[j+1][1] - dat[j][1]
        sn = dx1 * dx2 + dy1 * dy2
        d1 = math.sqrt(dx1*dx1+dy1*dy1)
        d2 = math.sqrt(dx2*dx2+dy2*dy2)
        if d1 != 0.0 and d2 != 0.0:
            sn /= d1 * d2
            if sn >= 0.8:
                doadd = 0
        if doadd:
            plst.addlist(dat[j])

print(len(plst.coord))

plst.plot(my_map)

my_map.save('debug.html')

print("Saved debug.html")

1 3843
2 5222
3 6175
4 6849
5 7348
6 7752
7 8106
8 8423
9 8672
10 8963
11 9134
12 9315
13 9517
14 9674
15 9795
16 9945
17 10067
18 10174
19 10284
20 10373
21 10477
22 10556
23 10673
24 10763
25 10843
26 10906
27 10978
28 11024
29 11068
30 11142
31 11187
32 11245
33 11301
34 11374
35 11446
36 11507
37 11558
38 11605
39 11640
40 11689
41 11736
42 11770
43 11811
44 11860
45 11914
46 11945
47 11977
48 12005
49 12049
50 12083
51 12130
52 12178
53 12225
54 12247
55 12285
56 12317
57 12352
58 12384
59 12406
60 12436
61 12470
62 12487
63 12508
64 12527
65 12551
66 12558
67 12574
68 12582
69 12607
70 12642
71 12660
72 12680
73 12694
74 12732
75 12760
76 12784
77 12811
78 12831
79 12855
80 12873
81 12886
82 12901
83 12917
84 12946
85 12966
86 12980
87 13008
88 13030
89 13059
90 13077
91 13105
92 13120
93 13136
94 13149
95 13170
96 13190
97 13202
98 13220
99 13235
100 13258
101 13288
102 13304
103 13320
104 13332
105 13356
106 13381
107 13400
108 13419
109 13441
110 13452
111 13462
112 13480
113 

In [50]:
pdf = pd.DataFrame(columns=['x', 'y'])

stat = 0

for i in range(plst.numPoints):
    if plst.cntr[i] > 4:
        pdf.loc[len(pdf)] = [plst.coord[i][0], plst.coord[i][1]]
        stat += 1

print(stat)

pdf.to_csv("points{}.csv".format(city), index=False)

10722


In [53]:
city = 1078
plst = PointList()

pdf = pd.read_csv("points{}.csv".format(city))

stat = 0

n = len(pdf)

for r in range(n):
    xy = [pdf.iloc[r].x, pdf.iloc[r].y]
    if (r % 10000 == 0):
        print(xy)
    plst.addlist(xy, 5)
    stat += 1
print(stat)


[55.82581999999832, 49.134040000002834]
[55.737719999999996, 49.14560000000001]
[55.80125, 49.23002]
20960


In [54]:
import folium
import random

# city = 338

start = citylist[city][0]

dat = pl.decode(df.iloc[start].route)
sp = dat[0]

numRoutes = len(citylist[city])


nbr = []
for i in range(plst.numPoints):
    nbr.append(set())

ncnt = [0] * plst.numPoints

for i in range(numRoutes):
    if i % 1000 == 0:
        print(i // 1000)
    ix = citylist[city][i]
    try:
        dat = pl.decode(df.iloc[ix].route)
    except TypeError:
        continue
    newRoute = []
    numPoints = len(dat)
    for j in range(0, numPoints):
        #if j == 0 or j == numPoints-1:
        #    newRoute.append(j)
        if j != 0:
            plst.getlinepoints(newRoute, dat[j-1], dat[j])
    l = len(newRoute)
    prev = -1
    for j in range(l):
        if (prev == newRoute[j]):
            continue
        if prev != -1:
            ncnt[prev] += 1
            nbr[prev].add(newRoute[j])
            nbr[newRoute[j]].add(prev)
        prev = newRoute[j]
    ncnt[prev] += 1





0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [70]:
# pdf = pd.DataFrame(columns=['id', 'p200', 'p500', 'p1000'])

#numTries = 100
my_map = folium.Map(sp, zoom_start=20)

csvdict = []

numRoutes = len(citylist_test[city])

print(numRoutes)

for i in range(200):
    if i % 1000 == 0:
        print(i // 1000)
    #if i % 50000 == 0:
    #    if dfcount != 0:
    #        pdf.to_csv("pr{}_{}".format(city, dfcount),index=False)
    #    print("Dataframe created")
    #    pdf = pd.DataFrame(columns=['id', 'p200', 'p500', 'p1000'])
    #    dfcount += 1
    ix = citylist_test[city][i]
    Id = df_test.iloc[ix].Id
    try:
        dat = pl.decode(df_test.iloc[ix].route)
    except TypeError:
        continue
    newRoute = []
    numPoints = len(dat)
    off = 0.0001
    prev = -1
    for j in range(0, numPoints):
        #if j == 0 or j == numPoints-1:
        #    newRoute.append(j)
        if j != 0:
            plst.getlinepoints(newRoute, dat[j-1], dat[j])
    folium.PolyLine(dat, color="red", weight=2.5, opacity=1).add_to(my_map)
    #newRouteData = []
    l = len(newRoute)
    prev = -1
    f = 0
    pc200 = 0
    pc500 = 0
    pc1000 = 0
    for j in range(l):
        if (prev == newRoute[j]):
            continue
        if len(nbr[prev]) > 2:
            if ncnt[prev] >= 200:
                pc200 += 1
            if ncnt[prev] >= 500:
                pc500 += 1
            if ncnt[prev] >= 1000:
                pc1000 += 1
            if ncnt[prev] >= 1000:
                folium.CircleMarker([plst.coord[prev][0], plst.coord[prev][1]], color='green').add_to(my_map)
        #if len(nbr[newRoute[j]]) > 2 and ncnt[newRoute[j]] > 500:
        #    f = 1
        #    newRouteData.append([plst.coord[newRoute[j]][0]+off, plst.coord[newRoute[j]][1]+off])
        prev = newRoute[j]
    if len(nbr[prev]) > 2:
        if ncnt[prev] >= 200:
            pc200 += 1
        if ncnt[prev] >= 500:
            pc500 += 1
        if ncnt[prev] >= 1000:
            pc1000 += 1
        if ncnt[prev] >= 1000:
            folium.CircleMarker([plst.coord[prev][0], plst.coord[prev][1]], color='green').add_to(my_map)
    csvdict.append({"id":Id, "p200":pc200, "p500":pc500, "p1000":pc1000})
    #pdf.loc[len(pdf)] = [ix, pc200, pc500, pc1000]
    #newRouteData.append([plst.coord[prev][0]+off, plst.coord[prev][1]+off])
    #if not f:
    #    continue
    #folium.PolyLine(newRouteData, color="cyan", weight=2.5, opacity=1).add_to(my_map)
    #for j in newRouteData:
    #    folium.CircleMarker(j, color='green').add_to(my_map)

print("done")

pdf = pd.DataFrame(csvdict)

# plst.plot(my_map)

my_map.save('debug2.html')


pdf.to_csv("pr{}.csv".format(city),index=False)

34922
0
done


In [6]:
print(citylist.keys())

dict_keys([1078, 22390, 22430, 22438])
